# Partitioning

* **Partitioning** is the splitting of data across multiple nodes.
* It is a necessity for scaling to particularly large datasets.
* It is usually combined with replication.


* The main problem that needs to be addressed by your partitioning scheme is the avoidance of **hot spots**.
* Data is not accessed or written to equally. If you partition too many commonly worked-with pieces of data to the same node, your performance will degrade.


## Partition types

* Depending on your goal, there are a couple of ways to partition data that are currently in use.
* The first is **partitioning by key range**. Partitioning by key range is simple: you just push contiguous strips of data to the same partitions.
* If there are hot spots in a particular key range, your performance will suffer.
* On the other hand, key range partitions support efficient range queries, as all of the data in a range will land on the same node. The other scheme does not provide this property.
* The alternative is **partitioning by key hash**.
* A good hashing function will uniformally distribute skewed data, "smudging" your data evenly across your partitions.
* This approach means no hot spots. However, you lose range query performace, as these must now hit all of the nodes.
* Cassandra has an interesting compromise position. It allows compound primary keys, and performs partitions by key hash based on the first key value only. All of the values from the second key element on will land on the same partition, so these keys can still be ranged over effectively!
* Note that even with the key hash technique you can still have hot spots, if your application has Zipf-like elements in it. For example, Twitter uses different serving techniques for celebrities then it does for "the rest of us".


## Partitioning secondary indices

* Partitioning support with secondary indices is another subject matter.
* Because of how difficult this is to implement, some data stores that support partitioning omit secondary indices entirely!
* Generally there are again two approaches.
* One is **scatter-gather**. The secondary indices are stored locally alongside the partitions. Requests that mean to use the secondary index must hit *all* of the nodes in order to stitch together the full index.
* This is also known as the **local index** approach.


* The alternative is a **global index**. The global index stores all of the information on all of the data.
* This global index isn't kept on one node, as doing so would create a hot spot and defeat the entire purpose of the thing. Strategies for storing it differ, but it's generally dealt with independently of data duplication.
* A local index requires hitting all or many of the partitions when performing an index-assisted read, but doesn't affect write performance. A global index requires hitting only a node containing the global index, but slow down writes, as these must now visit that node to notarize inserts.
* In practice global indices are updated asynchronously, which avoids this slowdown. But this introduces...all of those problems into the equation. Exciting.


## Rebalancing

* If you partition, you will eventually need to **rebalance**.
* Rebalancing schemes are all about accounting for the introduction of additional partitions into the store, whilst limiting the amount of actual data that needs to get moved around in response.
* E.g. a `hash mod N` would work poorly because it would result in a lot of data having to be moved.
* There are two popular rebalancing schemes.


* One is to have a **fixed partitions strategy**, where that number greatly exceeds the number of actual nodes. Cassandra calls these "virtual nodes" for example.
* When a new partition comes up, the partition grabs responsibility from some still-mostly-empty partitions from the existing partitions. Little data moves!
* This is an additional abstraction layer, however, and it requires operational overhead. The point is lost if you use too extreme a number of partitions.


* The laternative is **dynamic partitioning**. In this case you specify a maximum partition size, and split the partition when the data approaches that size (like growing a `dict` in Python).
* Partitions can stay on the same node after a split for a while, but will transfer over to a different node (snapshot operation). E.g. HBase implements this strategy using HDFS snapshotting.
* The advantage of this scheme is that it requires. The disadvantage is that it is ungainly when the datasets are small.


## Service discovery

* This chapter briefly mentions the service discovery problem, posing three possible solutions:
  * The client can connect to any node, which can forward the request through the network to the node that has the requested data.
  * A routing tier can connect to the necessary node.
  * The client can maintain information on the right node, and connect to it directly.
* Zookeeper is a hierarchical key-value store designed to store config metadata that is used by many services to address this exact problem.